In [3]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from functools import partial
from scipy import optimize
import scipy.stats as stats
import math
import scipy
from scipy.optimize import fmin

In [4]:
def generate_samples(n,mean_vec,cov_mat):
    """
    Generate n samples from multivariate Gaussian dist
    """
    Y = np.random.multivariate_normal(mean_vec,cov_mat,n)
    return Y

# Question 1

In [5]:
def max_likelihood(Y,n):

    s = np.zeros([2,2])
    for i in range(n):
        s = s + np.outer(Y[i],Y[i])
    s = s/n
    
    print("MLE: "+str(s))

# Question 2 and 3

In [19]:
def bayes_estim(Y,v_0,delta_0,n,prior="Wishart"):
    
    s = np.zeros([2,2])
    for i in range(n):
        s = s + np.outer(Y[i],Y[i])
    
    v_n = v_0 + n
    delta_n = delta_0 + s
    
    if(prior=="Wishart"):
        generator = stats.invwishart(df = v_n,scale=delta_n)
    if(prior=="Non_Info_Jeffrey"):
        v_n=1+n
        delta_n=0+s
        generator = stats.invwishart(df=v_n,scale=delta_n)
    if(prior=="Indep_Jeffrey"):
        v_n=0+n
        delta_n=0+s
        generator = stats.invwishart(df=v_n,scale=delta_n)
    

    mmse = generator.mean()
    mAP = generator.mode()

    print("\nBayes MMSE: " + str(mmse))
    print("\nBayes mAP: "+ str(mAP))

# Question 4

In [7]:
def bayes_monte_carlo(Y,v_0,delta_0,n,m):
    
    s = np.zeros([2,2])
    for i in range(n):
        s = s + np.outer(Y[i],Y[i])
    
    v_n = v_0 + n
    delta_n = delta_0 + s

    generator = stats.invwishart(df = v_n,scale=delta_n)
    
    numer = np.zeros([2,2])
    denom = 0
    
    for i in range(m):

        sigma = generator.rvs(size=1)

        numer = numer + (1/(i+1))*(sigma-numer)

        denom = denom + (1/(i+1))*(1-denom)
    
    print("\nMC integ: "+str(numer/denom))


# Question 5

In [8]:
def gibbs_sampler(Y,v_0,A_1,A_2,n):

    s = np.zeros([2,2])
    for i in range(n):
        s = s + np.outer(Y[i],Y[i])        

    v_n = v_0 + n
    
    a_1 = stats.invgamma.rvs(scale = 0.5, a = 1/(A_1)**2,size=1)
    a_2 = stats.invgamma.rvs(scale = 0.5, a = 1/(A_2)**2,size=1)
    a_1= 0.5
    a_2 = 0.5
    cov_mat = np.zeros([2,2])
    for k in range(1000):
        sigma = stats.invwishart.rvs(df=v_0+1+n,scale=2*v_0*np.diag([a_1,a_2])+s,size=1)
        cov_mat = cov_mat + (1/(k+1))*(sigma-cov_mat)
        
        a_1 = stats.invgamma.rvs(scale=(v_0+n)/2,a=v_0*np.linalg.inv(sigma)[0,0]+1/(A_1)**2,size=1)
        a_2 = stats.invgamma.rvs(scale=(v_0+n)/2,a=v_0*np.linalg.inv(sigma)[1,1]+1/(A_2)**2,size=1)

    print("\nGibbs estim: "+str(cov_mat))

# Question 6

In [9]:
def empirical_bayes(Y,v_0,n):
    s = np.zeros([2,2])
    for i in range(n):
        s = s + np.outer(Y[i],Y[i])
        
    nu_opt = optimize.minimize(equations,[v_0],bounds=[(1.001, 5000)])
    nu_opt=nu_opt.x[0]
    delta_opt=nu_opt/n*s
    
    
    generator = stats.invwishart(df = nu_opt+n,scale=delta_opt+s)
    

    mmse = generator.mean()
    mAP = generator.mode()
    print("\nBayes MMSE: " + str(mmse))
    print("\nBayes mAP: "+ str(mAP))
    

In [10]:
def multivariate_gamma(a,d):
    return scipy.special.multigammaln(a,d)

In [11]:
def equations(nu):
    #print(nu)
    return -(nu*np.log((nu+n)/nu)+n*(nu+n)/n+multivariate_gamma(nu/2,2)-multivariate_gamma((nu+n)/2,2))

## Experiments for n=1000

In [12]:
n = 1000
cov_mat = np.array([[1,0],[0,2]])
mean_vec = np.array([0,0])
v_0 = 5
delta_0 = np.array([[4,0],[0,5]])
m = 1000
A_1 = 0.05
A_2 = 0.05

In [13]:
Y=generate_samples(n,mean_vec,cov_mat)

In [20]:
max_likelihood(Y,n)
bayes_estim(Y,v_0,delta_0,n)
bayes_estim(Y,v_0,delta_0,n,prior="Non_Info_Jeffrey")
bayes_estim(Y,v_0,delta_0,n,prior="Indep_Jeffrey")
bayes_monte_carlo(Y,v_0,delta_0,n,m)
gibbs_sampler(Y,v_0,A_1,A_2,n)
empirical_bayes(Y,v_0,n)

MLE: [[1.02338679 0.04637075]
 [0.04637075 1.93440882]]

Bayes MMSE: [[1.02533612 0.04627819]
 [0.04627819 1.93553775]]

Bayes mAP: [[1.01923292 0.04600273]
 [0.04600273 1.92401669]]

Bayes MMSE: [[1.02543766 0.04646367]
 [0.04646367 1.93828539]]

Bayes mAP: [[1.01930955 0.046186  ]
 [0.046186   1.92670201]]

Bayes MMSE: [[1.02646619 0.04651028]
 [0.04651028 1.94022951]]

Bayes mAP: [[1.02032581 0.04623205]
 [0.04623205 1.92862295]]

MC integ: [[1.02525955 0.04979363]
 [0.04979363 1.93847193]]

Gibbs estim: [[1.02945399 0.05770923]
 [0.05770923 1.94655608]]

Bayes MMSE: [[1.0264631  0.04651014]
 [0.04651014 1.94022367]]

Bayes mAP: [[1.02032886 0.04623219]
 [0.04623219 1.92862872]]


## Experiments for n=100

In [21]:
n=100

In [23]:
max_likelihood(Y,n)
bayes_estim(Y,v_0,delta_0,n)
bayes_estim(Y,v_0,delta_0,n,prior="Non_Info_Jeffrey")
bayes_estim(Y,v_0,delta_0,n,prior="Indep_Jeffrey")
bayes_monte_carlo(Y,v_0,delta_0,n,m)
gibbs_sampler(Y,v_0,A_1,A_2,n)
empirical_bayes(Y,v_0,n)

MLE: [[ 1.02540292 -0.20723622]
 [-0.20723622  1.77015488]]

Bayes MMSE: [[ 1.04451267 -0.20317276]
 [-0.20317276  1.78446557]]

Bayes mAP: [[ 0.98648419 -0.19188539]
 [-0.19188539  1.6853286 ]]

Bayes MMSE: [[ 1.04632951 -0.21146553]
 [-0.21146553  1.80628049]]

Bayes mAP: [[ 0.98596435 -0.1992656 ]
 [-0.1992656   1.702072  ]]

Bayes MMSE: [[ 1.05711641 -0.21364559]
 [-0.21364559  1.82490194]]

Bayes mAP: [[ 0.99553682 -0.20120021]
 [-0.20120021  1.71859698]]

MC integ: [[ 1.05177889 -0.20575317]
 [-0.20575317  1.76491716]]

Gibbs estim: [[ 1.01022174 -0.18723492]
 [-0.18723492  1.72285557]]

Bayes MMSE: [[ 1.05679249 -0.21358012]
 [-0.21358012  1.82434275]]

Bayes mAP: [[ 0.99582427 -0.20125831]
 [-0.20125831  1.71909322]]


In [24]:
n=10

In [25]:
max_likelihood(Y,n)
bayes_estim(Y,v_0,delta_0,n)
bayes_estim(Y,v_0,delta_0,n,prior="Non_Info_Jeffrey")
bayes_estim(Y,v_0,delta_0,n,prior="Indep_Jeffrey")
bayes_monte_carlo(Y,v_0,delta_0,n,m)
gibbs_sampler(Y,v_0,A_1,A_2,n)
empirical_bayes(Y,v_0,n)

MLE: [[1.10028495 0.18197701]
 [0.18197701 1.96976728]]

Bayes MMSE: [[1.25023746 0.15164751]
 [0.15164751 2.0581394 ]]

Bayes mAP: [[0.83349164 0.10109834]
 [0.10109834 1.37209293]]

Bayes MMSE: [[1.37535619 0.22747126]
 [0.22747126 2.4622091 ]]

Bayes mAP: [[0.78591782 0.12998358]
 [0.12998358 1.40697663]]

Bayes MMSE: [[1.57183565 0.25996716]
 [0.25996716 2.81395325]]

Bayes mAP: [[0.84637304 0.13998232]
 [0.13998232 1.5152056 ]]

MC integ: [[1.23325878 0.14210141]
 [0.14210141 2.09194999]]

Gibbs estim: [[0.8483222  0.1661405 ]
 [0.1661405  1.55620297]]

Bayes MMSE: [[1.1009442  0.18208604]
 [0.18208604 1.97094749]]

Bayes mAP: [[1.0996265  0.18186811]
 [0.18186811 1.96858848]]


All methods perform similar when there is a lot of data available. Empirical Bayes is essentially MLE. So, low data Empirical Bayes/MLE, and when there is lot of data available Gibbs/empirical bayes